In [1]:
import os
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import gzip
import pickle as pkl
import pandas as pd

def loadall_results2(path, n_folds):
    regret = []
    history_actions = []
    history_outcomes = []
    perf = []
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds+1):
            try:
                data = pkl.load(f)
            except EOFError:
                break

            if type(data) == dict:
                regret.append( data['regret'] ) 
                history_actions.append( data['action_history'] )
                history_outcomes.append( data['outcome_history'] ) 
                perf.append( data['pred'] ) 

    return regret, history_actions, history_outcomes, perf

In [25]:
n_folds = 25
horizon = 9999
context = 'MNISTbinary'
model = 'MLP'
case = 'case1'
agent_name = 'EEneuralcbpside_v6' #ineural6, neuronal6

direct = './results/'
path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case,model,context,horizon,n_folds,agent_name) )
regret, action_history, outcome_history, perf = loadall_results2(path, n_folds)


In [2]:
def format_perf(perf, model, dataset, l_label):

    processed_data = []
    i = 0
    for item in perf:
        processed_item = {}
        # key_mapping = dict( zip( item.keys(), [10, 50, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 9000]))
        new_dict = item #{key_mapping[old_key]: value for old_key, value in item.items()}
        for key in new_dict:
            # processed_item[f'accuracy_{key}'] = new_dict[key]['accuracy']
            processed_item[f'f1_{key}'] = new_dict[key]['f1']
        processed_item['Seed'] = i
        processed_item['Model'] = model
        processed_item['Dataset'] = dataset
        processed_item['Approach'] = l_label
        i = i+1
        processed_data.append(processed_item)
    return processed_data


def convert_action_history(action_history, n_folds):
    counters = []
    for history in action_history:
        counter = np.cumsum([ 1 if action == 0 else 0 for action in history ])
        counters.append(counter)

    std = 2.575 * np.std(counters, axis = 0) / n_folds 
    # std = np.std(counters, axis = 0) / n_folds 

    return np.mean(counters, axis = 0), std 

n_folds = 26
horizon = 9999
model = 'MLP'

material = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    'ineural6': {'color': [51, 255, 255], 'label':'IneurAL (official)'},                    # Yellow
    'ineural3': {'color': [0, 0, 255], 'label':'IneurAL (tuned)'},                    # Cyan
    'neuronal6': {'color': [255, 0, 255], 'label':'Neuronal (official)'},                  # Magenta
    'neuronal3': {'color': [160, 160, 160], 'label':'Neuronal (tuned)'},                   # Orange
    'EEneuralcbpside_v6': {'color': [255, 0, 0], 'label': 'Neural-CBP'},  # Red
    'margin': {'color': [160, 160, 160], 'label':'Margin'},
    'cesa': {'color': [0, 0, 255], 'label':'Cesa'},
}

fig = go.Figure( )

data_models = {}
perfs_final = {}
explo_final = {}
explo2_final = {}

for model in ['MLP']:  # 'LeNet'

    if model == 'MLP':
        n_folds = 25
        datasets = ['MNISTbinary', 'adult', 'MagicTelescope' ] # 'covertype', 'shuttle',
    # else:
    #     datasets = ['MNIST', 'FASHION', 'CIFAR10',]
    #     n_folds = 25

    data_regrets = {} 
    data_perfs = {} 
    data_explo = {}
    data_explo2 = {}
    for data in datasets: 

        # if data in ['MNIST', 'FASHION', 'CIFAR10']:
        #     case = 'case2' 
        # elif data in ['covertype', 'shuttle']:
        #     case = 'game_case_seven'
        # else:
        case = 'case1'
        
        final_regrets = {}
        final_perfs = {}
        explo_perfs = {}
        explo2_perfs = {}
        for agent_name in material.keys():

            color, l_label = material[agent_name]['color'], material[agent_name]['label']

            r,g,b = color

            # try:
            direct = './results/'
            path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case, model, data, horizon,n_folds,agent_name) )
            print(path)
            regret, action_history,outcome_history, perf = loadall_results2(path, n_folds)
            regret = np.array(regret)
            print(regret)
            regret = regret#[1:]

            final_regrets[l_label] = regret[:,-1] 
            final_perfs[l_label] = pd.DataFrame( format_perf(perf, model, data, l_label) )

            m, s = convert_action_history(action_history, n_folds)
            explo_perfs[l_label] = m
            explo2_perfs[l_label] = s


        if data == 'MNISTbinary':
            data = 'MNISTbinary'
        if data == 'FASHION':
            data = 'FASHION'
            
        data_regrets[data] = final_regrets

        dataframes_list = list(final_perfs.values())
        concatenated_df = pd.concat(dataframes_list, axis=0)
        data_perfs[data] = concatenated_df

        data_explo[data] = explo_perfs
        data_explo2[data] = explo2_perfs
        
    data_models[model] = data_regrets
    dataframes_list = list(data_perfs.values())
    concatenated_df = pd.concat(dataframes_list, axis=0)

    explo_final[model] = data_explo
    explo2_final[model] = data_explo2


./results/case1_MLP_MNISTbinary_9999_25_ineural6.pkl.gz
[[1.000e+00 1.000e+00 2.000e+00 ... 1.588e+03 1.588e+03 1.589e+03]
 [1.000e+00 1.000e+00 1.000e+00 ... 1.659e+03 1.659e+03 1.660e+03]
 [1.000e+00 2.000e+00 2.000e+00 ... 1.651e+03 1.651e+03 1.651e+03]
 ...
 [1.000e+00 1.000e+00 1.000e+00 ... 1.616e+03 1.616e+03 1.616e+03]
 [1.000e+00 1.000e+00 2.000e+00 ... 1.674e+03 1.674e+03 1.674e+03]
 [1.000e+00 1.000e+00 1.000e+00 ... 1.637e+03 1.637e+03 1.637e+03]]
./results/case1_MLP_MNISTbinary_9999_25_ineural3.pkl.gz
[[1.000e+00 1.000e+00 2.000e+00 ... 1.492e+03 1.492e+03 1.492e+03]
 [1.000e+00 1.000e+00 1.000e+00 ... 1.410e+03 1.410e+03 1.410e+03]
 [1.000e+00 2.000e+00 2.000e+00 ... 1.424e+03 1.424e+03 1.424e+03]
 ...
 [1.000e+00 1.000e+00 1.000e+00 ... 1.356e+03 1.356e+03 1.356e+03]
 [1.000e+00 1.000e+00 2.000e+00 ... 1.259e+03 1.259e+03 1.259e+03]
 [1.000e+00 1.000e+00 1.000e+00 ... 2.014e+03 2.014e+03 2.014e+03]]
./results/case1_MLP_MNISTbinary_9999_25_neuronal6.pkl.gz
[[1.000e+00 1.0

In [3]:
import pandas as pd
# pd.DataFrame(data_regrets)
# data_regrets

samples = []
# Iterate over the dictionary to extract data
for model, dataset in data_models.items():
    for data, approach in dataset.items():
        for appr, values in approach.items():
            i = 0
            for val in values:
                # print(i, appr,val)
                samples.append([i, model, data, appr, val])
                i = i+1

# Create a DataFrame
df = pd.DataFrame(samples, columns=['Seed','Model', 'Dataset', 'Approach', 'Value'])
df = pd.merge(df, concatenated_df, on=['Model', 'Dataset', 'Approach', 'Seed'])


In [4]:
exclude_columns = ['Seed', 'Model', 'Dataset', 'Approach']

agg_columns = [col for col in df.columns if col not in exclude_columns]

df_mean = df.groupby(['Model', 'Dataset', 'Approach'])[agg_columns].agg(['mean',]) # 'std'
df_mean.columns = df_mean.columns.get_level_values(0)
df_mean = df_mean.reset_index()

df_std = df.groupby(['Model', 'Dataset', 'Approach'])[agg_columns].agg(['std',]) # 'std'
df_std.columns = df_std.columns.get_level_values(0)
df_std = df_std.reset_index()


In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

dataset = 'MagicTelescope'

approaches = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    'IneurAL (official)': {'color': [255, 161, 90], 'label':'IneurAL6'},                    # Yellow
    'IneurAL (tuned)': {'color': [25, 211, 243], 'label':'IneurAL3'},                    # Cyan
    'Neuronal (official)': {'color': [255, 102, 146], 'label':'neuronal6'},                  # Magenta
    'Neuronal (tuned)': {'color': [182, 232, 128], 'label':'neuronal3'},                   # Orange
    'Margin': {'color': [255, 151, 255], 'label':'margin'},
    'Cesa': {'color': [254, 203, 82], 'label':'cesa'},
    'Neural-CBP': {'color': [171, 99, 250], 'label': 'EEneuralcbpside_v6'},  # Red
}

thresholds = ['f1_10', 'f1_25', 'f1_50', 'f1_100', 'f1_150',
              'f1_250', 'f1_300', 'f1_400', 'f1_500', 'f1_750', 
              'f1_1000', 'f1_2500', 'f1_5000', 'f1_7500',]

# Create a subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=1,) #subplot_titles=("f1-score", "# of labelled observations")

# Add your traces for the first figure to the first subplot
for approach in approaches.keys():
    # ... [your existing code for calculating mean_values, std_values, and color] ...
    mean_values = [] 
    std_values = []
    r,g,b = approaches[approach]['color']
    
    for threshold in thresholds:
        mean_value = df_mean[(df_mean['Dataset'] == dataset) & (df_mean['Approach'] == approach)][threshold].values[0]
        mean_values.append(mean_value)

        std_value = df_std[(df_std['Dataset'] == dataset) & (df_std['Approach'] == approach)][threshold].values[0]
        std_values.append( 2.575 * std_value / n_folds  )
        # std_values.append(  std_value)

    fig.add_trace(go.Scatter(
        x=[10, 25, 50, 100, 150, 250, 300, 400, 500, 750, 1000, 2500, 5000, 7500, 9000], 
        y=mean_values,
        error_y=dict(type='data', array=std_values, visible=True, thickness=1, width=3),
        line=dict(dash='solid', width=1, color='rgba({},{},{},1)'.format(r,g,b)),
        marker=dict(size=2.5),
        name=approach), 
        row=1, col=1)

# Add your traces for the second figure to the second subplot
# for approach in approaches.keys():
#     # ... [your existing code for calculating explo_value, upper, and color] ...
#     r,g,b = approaches[approach]['color']
#     explo_value = explo_final['MLP'][dataset][approach]
#     upper = explo_value + explo2_final['MLP'][dataset][approach]
#     xcoords = np.arange(0,horizon,1).tolist()

#     fig.add_trace(go.Scatter(
#         x=xcoords+xcoords[::-1], 
#         y=upper.tolist()+explo_value.tolist()[::-1],  
#         fill='toself',
#         fillcolor='rgba({},{},{},0.2)'.format(r,g,b),
#         line=dict(color='rgba({},{},{},1)'.format(r,g,b), width=0),
#         hoverinfo="skip"),
#         row=1, col=2)
    
#     fig.add_trace(go.Scatter(
#         x=xcoords, 
#         y=explo_value,
#         line=dict(dash='solid', width=2, color='rgba({},{},{},1)'.format(r,g,b)),
#         name=approach), 
#         row=1, col=2)

# Update layout and axes for the combined figure
siz = 12.5
fig.update_layout(
    width=220,  # Adjusted for two subplots
    height=180,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=False
)

# Update xaxis and yaxis properties for each subplot
fig.update_xaxes(title_text="Expert query volume", row=1, col=1, gridcolor='lightgrey', tickmode='array', type="log", title_standoff=5, title_font=dict(size=siz-2), tickfont=dict(size=siz-2))
fig.update_yaxes(title_text="f1-score ± 99% CI",  row=1, col=1, gridcolor='lightgrey', #range=[0,1.05]
                 title_standoff=5, title_font=dict(size=siz-2), tickfont=dict(size=siz-2), tickvals=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] )

# fig.update_xaxes(title_text="Round", row=1, col=2, gridcolor='lightgrey', tickmode='array', title_standoff=5, title_font=dict(size=siz-2), tickfont=dict(size=siz-2))
# fig.update_yaxes(title_text="# of labels ± 99% CI", range=[0,4.5], row=1, col=2, gridcolor='lightgrey',type="log", title_standoff=0, title_font=dict(size=siz-2), tickfont=dict(size=siz-2))

fig.show()

# Save the combined figure as a PDF
# fig.write_image("./figures/combined_{}_{}_{}.pdf".format(case, model, dataset))
fig.write_image("./figures/tst.pdf".format(case, model, dataset))
fig.write_image("./figures/f1_{}_{}_{}.pdf".format(case, model, dataset))


In [87]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

dataset = 'MagicTelescope'

approaches = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'labeMNISTbinaryl': 'EEneuralcbpside_v5'},  # Red
    'IneurAL (official)': {'color': [255, 161, 90], 'label':'IneurAL6'},                    # Yellow
    'IneurAL (tuned)': {'color': [25, 211, 243], 'label':'IneurAL3'},                    # Cyan
    'Neuronal (official)': {'color': [255, 102, 146], 'label':'neuronal6'},                  # Magenta
    'Neuronal (tuned)': {'color': [182, 232, 128], 'label':'neuronal3'},                   # Orange
    'Margin': {'color': [255, 151, 255], 'label':'margin'},
    'Cesa': {'color': [254, 203, 82], 'label':'cesa'},
    'Neural-CBP': {'color': [171, 99, 250], 'label': 'EEneuralcbpside_v6'},  # Red
}

thresholds = ['f1_10', 'f1_25', 'f1_50', 'f1_100', 'f1_150',
              'f1_250', 'f1_300', 'f1_400', 'f1_500', 'f1_750', 
              'f1_1000', 'f1_2500', 'f1_5000', 'f1_7500',]

# Create a subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=1, ) #

# Add your traces for the first figure to the first subplot
for approach in approaches.keys():
    # ... [your existing code for calculating mean_values, std_values, and color] ...
    mean_values = [] 
    std_values = []
    r,g,b = approaches[approach]['color']
    
    for threshold in thresholds:
        mean_value = df_mean[(df_mean['Dataset'] == dataset) & (df_mean['Approach'] == approach)][threshold].values[0]
        mean_values.append(mean_value)

        std_value = df_std[(df_std['Dataset'] == dataset) & (df_std['Approach'] == approach)][threshold].values[0]
        std_values.append(std_value)

    fig.add_trace(go.Scatter(
        x=[10, 25, 50, 100, 150, 250, 300, 400, 500, 750, 1000, 2500, 5000, 7500, 9000], 
        y=mean_values,
        error_y=dict(type='data', array=std_values, visible=True, thickness=1, width=3),
        line=dict(dash='solid', width=2, color='rgba({},{},{},1)'.format(r,g,b)),
        marker=dict(size=2.5),
        name=approach), 
        row=1, col=1)

# Add your traces for the second figure to the second subplot
# for approach in approaches.keys():
#     # ... [your existing code for calculating explo_value, upper, and color] ...
#     r,g,b = approaches[approach]['color']
#     explo_value = explo_final['MLP'][dataset][approach]
#     upper = explo_value + explo2_final['MLP'][dataset][approach]
#     xcoords = np.arange(0,horizon,1).tolist()

#     fig.add_trace(go.Scatter(
#         x=xcoords+xcoords[::-1], 
#         y=upper.tolist()+explo_value.tolist()[::-1],  
#         fill='toself',
#         fillcolor='rgba({},{},{},0.2)'.format(r,g,b),
#         showlegend=False,
#         line=dict(color='rgba({},{},{},1)'.format(r,g,b), width=0),
#         hoverinfo="skip"),
#         row=1, col=2)
    
#     fig.add_trace(go.Scatter(
#         x=xcoords, 
#         y=explo_value,
#         showlegend=False,
#         line=dict(dash='solid', width=2, color='rgba({},{},{},1)'.format(r,g,b)),
#         name=approach), 
#         row=1, col=2)

# Update layout and axes for the combined figure
siz = 12.5
fig.update_layout(
    width=220,  # Adjusted for two subplots
    height=180,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=0, r=0, t=3, b=0),
    showlegend=True,
    # xaxis=dict(showgrid=False, showticklabels=False, zeroline=False, visible=False),  # Set visible to False
    # yaxis=dict(showgrid=False, showticklabels=False, zeroline=False, visible=False),  # Set visible to False

    legend=dict(
        orientation="v",
        yanchor="middle",
        y=0.5,  # Adjust this value to position the legend
        xanchor="center",
        x=0.5,
        bgcolor='rgba(0,0,0,0)',
        font=dict(size=siz),  # Increase legend font size
        # title=dict(
        #     text='<b>Legend</b>',  # Set the text for your legend title
        #     font=dict(size=siz)  # Optionally set the font size for the legend title
        # )
    )
)

# Update xaxis and yaxis properties for each subplot
fig.update_xaxes(title_text="Round", row=1, col=1, gridcolor='lightgrey', tickmode='array', type="log", title_standoff=5, title_font=dict(size=siz-2), tickfont=dict(size=siz-2))
fig.update_yaxes(title_text="f1-score", row=1, col=1, gridcolor='lightgrey', title_standoff=5, title_font=dict(size=siz-2), tickfont=dict(size=siz-2))

# fig.update_xaxes(title_text="Round", row=1, col=2, gridcolor='lightgrey', tickmode='array', title_standoff=5, title_font=dict(size=siz-2), tickfont=dict(size=siz-2))
# fig.update_yaxes(title_text="# of labelled observations", row=1, col=2, gridcolor='lightgrey',type="log", title_standoff=0, title_font=dict(size=siz-2), tickfont=dict(size=siz-2))

fig.show()

# Save the combined figure as a PDF
fig.write_image("./figures/binary_legend.png".format(case, model, dataset), scale=10)


In [63]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ... [your existing setup code for 'approaches' dictionary]

# Create an empty subplot
fig = make_subplots(rows=1, cols=1)

# Add your traces only for the purpose of generating the legend
for approach, info in approaches.items():
    r, g, b = info['color']
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(color='rgba({},{},{},1)'.format(r, g, b)),
        name=info['label']
    ))

# Update layout and axes for the figure
fig.update_layout(
    width=220,
    height=220,
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="middle",
        y=0.55,  # Adjust if necessary to position the legend
        xanchor="center",
        x=0.5,
        font=dict(size=12.5)
    ),
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False, visible=False),  # Set visible to False
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False, visible=False),  # Set visible to False
    margin=dict(l=0, r=0, t=0, b=0)
)

# Show the figure
fig.show()

# Save the figure with only the legend as a PDF
fig.write_image("./figures/binary_legend.pdf")
